In [1]:
from model import model
from model import *

import data_loader
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
import torch

%load_ext autoreload
%autoreload 2

Data iteration testing

In [2]:
frames = data_loader.get()
imgs= next(iter(frames))

In [3]:
imgs[0].shape

torch.Size([4, 3, 128, 128])

RDN

In [4]:
c = model.conv_feature_extractor.RDN(3,5,5,3,2)

In [5]:
feats=c(imgs[0])

Create graph from features

In [6]:
imsize = imgs[0].shape[-1]

In [7]:
grids = torch.cat([x.unsqueeze(0) for x in torch.meshgrid([torch.linspace(-1, 1, imsize) for _ in range(2)])])

In [8]:
batch_coords = grids.unsqueeze(0).repeat(4,*[1]*len(grids.shape))

In [9]:
batch_coords_added = torch.cat([batch_coords,feats],axis=1).shape

In [10]:
# Maps x,y to index in flattened list
flat_indices = np.arange(imsize**2).reshape((imsize,imsize)) 
# Create flattened adjacency list
adj_list = [[flat_indices[x,y],flat_indices[neighbx,neighby]]
                # For (very cell
                for x,y in [(x,y) for x in range(imsize) for y in range(imsize)]
                    # Every valid neighbor of current cell
                    for neighbx,neighby in [(x+xd,y+yd) for xd in range(-1,2) for yd in range(-1,2) 
                                   if -1<x+xd<imsize and -1<y+yd<imsize] ]
edge_tensor = torch.tensor(adj_list).t().contiguous()

In [36]:
from torch_geometric.data import Data,Batch

In [37]:
flat_feats = torch.flatten(feats,2,3).permute(0,2,1)

In [38]:
b_data=Data(x=flat_feats,edge_index=edge_tensor)

In [39]:
b_data.x.shape

torch.Size([4, 16384, 5])

In [40]:
bb_data=Batch.from_data_list([Data(b_data.x[i],b_data.edge_index) for i in range(b_data.x.shape[0])])

In [41]:
bb_data.x.shape

torch.Size([65536, 5])

Graph module testing

In [92]:
from model import graph_bottleneck

In [93]:
gb=graph_bottleneck.ObjGraph(5,2)

In [94]:
out_graph=gb(bb_data)

RuntimeError: Invalid index in scatterAdd at /pytorch/aten/src/TH/generic/THTensorEvenMoreMath.cpp:721
The above operation failed in interpreter.
Traceback (most recent call last):
  File "/home/cos008/.local/lib/python3.7/site-packages/torch_scatter/scatter.py", line 22
            size[dim] = int(index.max()) + 1
        out = torch.zeros(size, dtype=src.dtype, device=src.device)
        return out.scatter_add_(dim, index, src)
               ~~~~~~~~~~~~~~~~ <--- HERE
    else:
        return out.scatter_add_(dim, index, src)


In [88]:
out_graph[0].shape,out_graph[1].shape

(torch.Size([23060, 5]), torch.Size([23060]))

In [96]:
torch_geometric.utils.to_dense_batch(out_graph[0],out_graph[1])[0].shape

torch.Size([4, 5783, 5])

In [53]:
tmp=Data(x=out_graph[0],edge_index=out_graph[1])
tmp2=Batch([tmp])

In [58]:
import torch_geometric.utils.to_dense_batch

In [60]:
dense=torch_geometric.utils.to_dense_batch(out_graph[0],out_graph[2])

In [65]:
dense[1]

tensor([[False, False,  True,  ...,  True,  True,  True],
        [ True, False,  True,  ...,  True,  True,  True],
        [ True, False,  True,  ...,  True,  True,  True],
        [False,  True, False,  ..., False, False, False]])

Hypernetwork generation testing

In [27]:
from model import hypernet_img_gen

In [28]:
hp=hypernet_img_gen.HyperNetImgGen(128,5,12).cuda()

In [29]:
phi=hp(b_data.x)

RuntimeError: CUDA out of memory. Tried to allocate 16.06 GiB (GPU 0; 10.92 GiB total capacity; 620.38 MiB already allocated; 9.24 GiB free; 1.11 GiB reserved in total by PyTorch)

In [ ]:
phi[0].reshape(imsize,imsize,3)

Global module testing

In [25]:
from model.graph_bottleneck import ObjGraph

In [27]:
graph_bot = ObjGraph(5)

In [30]:
graph_vec = graph_bot(test)